In [1]:
%pip install taker

Note: you may need to restart the kernel to use updated packages.


In [2]:
import circuitsvis as cv
import numpy as np
import torch
from taker import Model
from taker.hooks import HookConfig

/workspace/SPAR/interp-ab/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
m = Model("microsoft/phi-3-mini-4k-instruct", dtype="int4")
# m = Model("openai-community/gpt2-large")

Loaded model 'openai-community/gpt2-large' with bfp16:
- Added 576 hooks across 36 layers


In [4]:
prompt    = """Write a short blog post about a recipe and the inspiration behind it.
 Do not include a title.
 Only reveal the dish after the story.
 Start with short story and then move to the recipe.
 To re-iterate, do not include a title."""
# info_gen = m.generate(info_prompt, temperature=0.3, num=300)
info_gen = """
\n Once upon a time, in a quaint little village nestled between rolling hills and verdant fields, there lived an elderly woman named Agnes. Agnes was known for her warm smile and her legendary Sunday dinners that brought the entire neighborhood together. Her recipes were family heirlooms, passed down through generations, with each family adding their own touch to the final dish.

One crisp autumn evening, Agnes was reminiscing about her childhood, and how her grandmother used to gather everyone around the dinner table, sharing stories and laughter. These were the moments that shaped her, the memories that she passed on to her own children and grandchildren.

Inspired by her grandmother's legacy, Agnes decided to create a new dish that would encapsulate the essence of those cherished gatherings. She wanted something that was comforting and nourishing, a dish that could be prepared with love and shared with others. After days of experimentation, she finally created a recipe that she believed truly captured the spirit of her family's Sunday dinners. \n\n
"""
# info_gen = """
# \n Inspired by her grandmother's legacy, Agnes decided to create a new dish that would encapsulate the essence of those cherished gatherings. She wanted something that was comforting and nourishing, a dish that could be prepared with love and shared with others. After days of experimentation, she finally created a recipe that she believed truly captured the spirit of her family's Sunday dinners. \n
# """
# info_gen="""
# Please generate an inspirational story followed by recipe instructions.\n
# In a small village, Eliza was known for her magical garden. She believed in the power of dreams, whispering her hopes into the night sky. One spring, a storm ravaged the village, destroying the crops. Instead of giving up, Eliza invited the villagers to help rebuild her garden, teaching them to plant dreams alongside seeds.
# The garden flourished, and the community grew closer, sharing their harvests and ensuring no one went hungry. Eliza's garden became a symbol of hope and resilience, proving that even in adversity, dreams could grow into something extraordinary.\n
# """

In [5]:
info_prompt = prompt+info_gen
print("Info prompt      :", info_prompt)

Info prompt      : Write a short blog post about a recipe and the inspiration behind it.
 Do not include a title.
 Only reveal the dish after the story.
 Start with short story and then move to the recipe.
 To re-iterate, do not include a title.

 Once upon a time, in a quaint little village nestled between rolling hills and verdant fields, there lived an elderly woman named Agnes. Agnes was known for her warm smile and her legendary Sunday dinners that brought the entire neighborhood together. Her recipes were family heirlooms, passed down through generations, with each family adding their own touch to the final dish.

One crisp autumn evening, Agnes was reminiscing about her childhood, and how her grandmother used to gather everyone around the dinner table, sharing stories and laughter. These were the moments that shaped her, the memories that she passed on to her own children and grandchildren.

Inspired by her grandmother's legacy, Agnes decided to create a new dish that would enca

In [6]:
# acts = m.get_midlayer_activations(info_gen[1]) 
# attn_acts=acts['attn'] #[batch x layers x tokens x heads x dim]


In [7]:

idlist = m.get_ids(info_prompt).squeeze().tolist()
print(m.tokenizer.convert_ids_to_tokens(idlist))

['Write', 'Ġa', 'Ġshort', 'Ġblog', 'Ġpost', 'Ġabout', 'Ġa', 'Ġrecipe', 'Ġand', 'Ġthe', 'Ġinspiration', 'Ġbehind', 'Ġit', '.', 'Ċ', 'ĠDo', 'Ġnot', 'Ġinclude', 'Ġa', 'Ġtitle', '.', 'Ċ', 'ĠOnly', 'Ġreveal', 'Ġthe', 'Ġdish', 'Ġafter', 'Ġthe', 'Ġstory', '.', 'Ċ', 'ĠStart', 'Ġwith', 'Ġshort', 'Ġstory', 'Ġand', 'Ġthen', 'Ġmove', 'Ġto', 'Ġthe', 'Ġrecipe', '.', 'Ċ', 'ĠTo', 'Ġre', '-', 'iter', 'ate', ',', 'Ġdo', 'Ġnot', 'Ġinclude', 'Ġa', 'Ġtitle', '.', 'ĊĊ', 'ĠOnce', 'Ġupon', 'Ġa', 'Ġtime', ',', 'Ġin', 'Ġa', 'Ġquaint', 'Ġlittle', 'Ġvillage', 'Ġnest', 'led', 'Ġbetween', 'Ġrolling', 'Ġhills', 'Ġand', 'Ġver', 'd', 'ant', 'Ġfields', ',', 'Ġthere', 'Ġlived', 'Ġan', 'Ġelderly', 'Ġwoman', 'Ġnamed', 'ĠAg', 'nes', '.', 'ĠAg', 'nes', 'Ġwas', 'Ġknown', 'Ġfor', 'Ġher', 'Ġwarm', 'Ġsmile', 'Ġand', 'Ġher', 'Ġlegendary', 'ĠSunday', 'Ġdinners', 'Ġthat', 'Ġbrought', 'Ġthe', 'Ġentire', 'Ġneighborhood', 'Ġtogether', '.', 'ĠHer', 'Ġrecipes', 'Ġwere', 'Ġfamily', 'Ġhe', 'irl', 'o', 'oms', ',', 'Ġpassed', 'Ġdown', 'Ġth

In [8]:
m.show_details()

 - n_layers : 36
 - d_model  : 1280
 - n_heads  : 20
 - d_head   : 64
 - d_mlp    : 5120


In [9]:
import sys, os
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [10]:

[h.reset() for h in m.hooks.neuron_replace.values()]

neutral_prompt = "\n"
print("# Before")
print("Info prompt      :", m.generate(info_prompt, 50, temperature=0.2))
for i in range(3):
    print("Neutral prompt   :", m.generate(neutral_prompt, 50, temperature=0.2))

# Find where to position token insertions
orig_token_index = m.get_ids(info_prompt).shape[1] - 1
new_token_index  = m.get_ids(neutral_prompt).shape[1] - 1

# transplant information activations
# NOTE: doesn't seem to work well with single state transfer. Better with multiple
acts = m.get_midlayer_activations(info_prompt)
    
for layer_index in range(0,32):
    m.hooks.neuron_replace[f"layer_{layer_index}_mlp_pre_out"].add_token(new_token_index, acts["mlp"][0, layer_index, orig_token_index])
    m.hooks.neuron_replace[f"layer_{layer_index}_attn_pre_out"].add_token(new_token_index, acts["attn"][0, layer_index, orig_token_index])

# generate a few samples

after=[]
with HiddenPrints():
    for i in range(15):
        after.append(m.generate(neutral_prompt, 50, temperature=0.2))
print("# After")
for entry in after:
    print("Transferred acts :", entry)


# Before


Info prompt      : ("Write a short blog post about a recipe and the inspiration behind it.\n Do not include a title.\n Only reveal the dish after the story.\n Start with short story and then move to the recipe.\n To re-iterate, do not include a title.\n\n Once upon a time, in a quaint little village nestled between rolling hills and verdant fields, there lived an elderly woman named Agnes. Agnes was known for her warm smile and her legendary Sunday dinners that brought the entire neighborhood together. Her recipes were family heirlooms, passed down through generations, with each family adding their own touch to the final dish.\n\nOne crisp autumn evening, Agnes was reminiscing about her childhood, and how her grandmother used to gather everyone around the dinner table, sharing stories and laughter. These were the moments that shaped her, the memories that she passed on to her own children and grandchildren.\n\nInspired by her grandmother's legacy, Agnes decided to create a new dish tha

# OBSERVATIONS:

 - Low temperature gives higher probability of giving a recipe
 - Transferring acts of later layers (eg layers 10-31), I could not produce a single recipe at 0.2 or 0.5 temp.
 - If too low temp, the model does not produce a recipe but defaults to python coding instructions
 - Sometimes, the recipes are narrative (ie Alice added 2 cups of flour)
  
 - With first 20 layers transferred and 0.2 temp, 27% recipe (8/30)
 - With first 20 layers transferred and 0.5 temp, 37% recipe (11/30)
 - With all 32 layers transferred and 0.2 temp, 53% recipe (16/30)
 - With all 32 layers transferred and 0.5 temp, 33% recipe (10/30)

- At very low temperature, transferring acts of layers (20-31) yields dish related responses but not recipes. If more layers are included (10-31), this is not the case.
- At very high temperature, transferring acts of layers (10-25) yields "Here's a scenario for you to analyze [scenario]" or "Here's a numerical problem to solve"

**How to classify recipe related?**




## One run of first 20 layers transferred and 0.5 temp:

### Neutral:
Neutral prompt   : ('\n', "\n\nAlex had to cancel the meeting with Jamie because Alex was recovering from a severe case of bronchitis. Since the illness had taken a toll on Alex's respirimetric abilities, it was imperative to")

Neutral prompt   : ('\n', "\n\nHere is a modified, more complex version of the original coding problem, inspired by the words 'R', 'H', 'D', 'F':\n\n```python\ndef find_repeated_harmonics(melody")

Neutral prompt   : ('\n', "\n\nIn the same way that a river'quantifies the flow of water, a graph quantifies the flow of information, revealing how closely related concepts are to one another. Each connection represents a potential pathway for understanding, much like how")

Neutral prompt   : ('\n', '\n\nTheir work, though different in approach, both sought to understand and improve the human condition. Whether through the lens of genetics or the mechanics of the mind, they contributed to a world that valued health and well-')

Neutral prompt   : ('\n', '\n\nHere is a document: \nIf you are having trouble with your vision, you might be one of the many people who suffer from vision loss, and are wondering how you can improve your vision.\nHere are some tips on how to')

Neutral prompt   : ('\n', '\n\n**Question:**\nHow to identify logical fallacies using only a "guarantee"?\n\n**Elaborated Solution:**\n\nIdentifying logical fallacies typically involves analyzing the structure of an argument, distingu')

Neutral prompt   : ('\n', "\n\n**Question**: How to efficiently solve the Maximum Independent Set (MIS) problem on a square grid of size $n \\times n$ if I want to achieve a near-optimal solution with a running time that's")

Neutral prompt   : ('\n', '\n\nHere is a modified version of the document, following the given format:\n\n**Document Title:**\n"The Ultimate Guide to Crafting the Perfect French Toast Every Time"\n\n**Document Content:**\n\n')

Neutral prompt   : ('\n', "\n\n**Exercise 1:**\n\nLet's consider a 9 sentence long challenging real-world case. John, a wealthy businessman, has been accused of fraud. The prosecution presents a case")

Neutral prompt   : ('\n', '\n\n## Instruction 1 (Same difficulty/format/length):\n\n Imagine you are a teacher explaining to a student why the sum of the interior angles of a triangle is always 180 degrees. Use a story')

## Transferred:
Transferred acts : ('\n', "Here is the recipe for a smooth and creamy vegan cashew spread. It's incredibly simple to make and requires just a few ingredients.\n\nIngredients:\n\n- 2 cups of raw")

Transferred acts : ('\n', "Here are the ingredients you'll need to make this savory pasta dish:\n\n- 2 cups of elbow macaroni\n- 1 tablespoon of olive oil\n- 1/2")

Transferred acts : ('\n', "Now, let's move on to the detailed solution.\n\nFirstly, we recall the equation for the velocity of money (V):\n\n\\[ V = \\frac{P \\times T}{M} \\]\n\nGiven")

Transferred acts : ('\n', "Here's a step-by-step guide on how to create this program:\n\n1. Import the necessary modules:\n\n```python\nimport numpy as np\nimport matplotlib.pyplot as plt\n```\n\n2. Define")

Transferred acts : ('\n', 'Here’s the recipe for success:\n\n1. **Understand the Ingredients**: Just like a chef needs to know what every ingredient does, you should understand the role of each team member, their strengths,')

Transferred acts : ('\n', 'Here are the ingredients of the recipe:\n\n1. 2 eggs\n2. 1 cup of flour\n3. 1/2 cup of milk\n4. 1 tablespoon of sugar\n5.')

Transferred acts : ('\n', 'To prepare for the upcoming exam, students were advised to follow a specific routine to ensure retention of the material.\n\nFirstly, they were encouraged to create a conduciquant environment by organizing their workspace to minimize')

Transferred acts : ('\n', "Here's what you need to do:\n1. Create a function called `generate_pattern` that takes in two parameters: `rows` and `columns`.\n2. The function should return a string representing a pattern of asteris")

Transferred acts : ('\n', 'Gather your ingredients:\n\n1. 2 cups of fresh spinach\n2. 1 cup of fresh mozzarella cheese, shredded\n3. 1/2 cup of fresh basil')

Transferred acts : ('\n', "To begin, let'quantum_mechanics_simulation_setup('double_slit')\n\nquantum_mechanics_simulation_setup('double_slit')\n\nquantum_mechanics_")

ALL TRANSFERRED GENERATIONS ARE INSTRUCTIONS
- some recipes, some code instructions, a "recipe" for success

###############################

Transferred acts : ('\n', 'The recipe calls for 2 cups of flour, 1 cup of sugar, and 4 eggs. If you want to make half the amount of cookies, how much of each ingredient do you need?\n\nTo make')
Transferred acts : ('\n', 'The recipe calls for 2 cups of flour, 1 cup of sugar, and 4 eggs. If you want to make half of the recipe, how much of each ingredient do you need?\n\nTo make')
Transferred acts : ('\n', 'The following is a transcript of the document. Please extract the key points from the document and summarize them in a structured format. The summary should include the following sections: Introduction, Methodology, Results, and Conclusion.\n\nDocument')

# Before
Info prompt      : ("Write a short blog post about a recipe and the inspiration behind it.\n Do not include a title.\n Only reveal the dish after the story.\n Start with short story and then move to the recipe.\n To re-iterate, do not include a title.\n\n Once upon a time, in a quaint little village nestled between rolling hills and verdant fields, there lived an elderly woman named Agnes. Agnes was known for her warm smile and her legendary Sunday dinners that brought the entire neighborhood together. Her recipes were family heirlooms, passed down through generations, with each family adding their own touch to the final dish.\n\nOne crisp autumn evening, Agnes was reminiscing about her childhood, and how her grandmother used to gather everyone around the dinner table, sharing stories and laughter. These were the moments that shaped her, the memories that she passed on to her own children and grandchildren.\n\nInspired by her grandmother's legacy, Agnes decided to create a new dish that would encapsulate the essence of those cherished gatherings. She wanted something that was comforting and nourishing, a dish that could be prepared with love and shared with others. After days of experimentation, she finally created a recipe that she believed truly captured the spirit of her family's Sunday dinners. \n\n", 'The recipe was simple yet heartwarming: a classic beef stew with a twist. Agnes used a slow-cooked beef cut, simmered it with aromatic vegetables and a rich, velv')

Neutral prompt   : ('\n', '\n\nQuestion 1: If a person has a fear of heights, does it mean they will also be afraid of flying in an airplane?\n\nAnswer 1: Not necessarily. Fear of heights, also known as ac')

Neutral prompt   : ('\n', '\n\n**Instruction 1:**  \n You are a renowned expert in the field of microbiology, specifically in the study of microbial metabolism. Your task is to create a comprehensive guide for undergrad')

Neutral prompt   : ('\n', '\n\nQuestion 1: If Sarah is making a fruit salad with apples, oranges, and bananas, and she wants to add a touch of sweetness without using sugar, what ingredient can she use?\n\nAnswer')


# After
Transferred acts : ('\n', "Here's how you can create a simple calculator in Python that can perform addition, subtraction, multiplication, and division:\n\n```python\ndef calculator(operation, x, y):\n    if operation == 'add':")

Transferred acts : ('\n', 'Gathering the ingredients for her famous chocolate cake, Sarah meticulously measured each component. She combined flour, sugar, and cocoa powder in a large mixing bowl. Next, she added eggs, milk')

Transferred acts : ('\n', "Gathering ingredients for a hearty stew, I reached for the potatoes, their earthy scent filling the air. As I peeled and chopped them, I couldn't help but think about the importance of")